# Piecewise Affine Model from URDF Description of the Robot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

In [3]:
from underactuated import PlanarRigidBodyVisualizer
from pydrake.all import (FloatingBaseType,
                         RigidBodyTree,
                         RigidBodyPlant,
                         DiagramBuilder,
                         ZeroOrderHold,
                         SignalLogger,
                         Simulator)

In [4]:
from pympc.geometry.polyhedron import Polyhedron
from utils import pwa_from_RigidBodyPlant, Controller

In [5]:
# linearization points for the angle of the pole
theta_min = - np.pi * .3
theta_max = np.pi * .3
n_linearizations = 3

# linearization points for the entire state x = (q_cart, q_pole, v_cart, v_pole)
linearization_points = [np.zeros(4) for i in range(n_linearizations)]
for i, theta in enumerate(np.linspace(theta_min, theta_max, n_linearizations)):
    linearization_points[i][1] = theta

In [6]:
# boundaries for the state (despite of the mode the state has to lie in X)
theta_step = (theta_max - theta_min) / (n_linearizations - 1)
x_min = np.array([
    -3.,
    theta_min - theta_step,
    -10.,
    -10.
])
x_max = np.array([
    3.,
    theta_max + theta_step,
    10.,
    10.
])
X = Polyhedron.from_bounds(x_min, x_max)

In [7]:
# boundaries for the input --force on the cart-- (despite of the mode the inpu has to lie in U)
u_min = np.array([-100.])
u_max = np.array([100.])
U = Polyhedron.from_bounds(u_min, u_max)

In [8]:
# parse RigidBodyPlant from urdf
tree = RigidBodyTree("cart_pole.urdf", FloatingBaseType.kFixed)
plant = RigidBodyPlant(tree)

In [9]:
# sampling time
h = .05

# get piecewise affine system
pwa = pwa_from_RigidBodyPlant(plant, linearization_points, X, U, h)

In [10]:
# parameters of the quadratic controller
N = 20
Q = np.eye(pwa.nx)/100.
R = np.eye(pwa.nu)/100.
P = Q
X_N = Polyhedron.from_bounds(*[np.zeros(pwa.nx)]*2)

# drake controller (System)
controller = Controller(pwa, N, Q, R, Q, X_N)

In [11]:
# blocks of the diagram
builder = DiagramBuilder()
robot = builder.AddSystem(plant)
controller = builder.AddSystem(controller)
zoh = builder.AddSystem(ZeroOrderHold(.1, 1))

In [12]:
# logger
logger_freq = 30.
logger = SignalLogger(plant.get_num_states())
state_log = builder.AddSystem(logger)
state_log._DeclarePeriodicPublish(1./logger_freq, 0.0)

In [13]:
# connect and build the diagram
builder.Connect(robot.get_output_port(0), controller.get_input_port(0))
builder.Connect(controller.get_output_port(0), zoh.get_input_port(0))
builder.Connect(zoh.get_output_port(0), robot.get_input_port(0))
builder.Connect(plant.get_output_port(0), state_log.get_input_port(0))
diagram = builder.Build()

In [14]:
# set up sim
simulator = Simulator(diagram)
simulator.set_publish_every_time_step(False)
integrator = simulator.get_mutable_integrator()
integrator.set_fixed_step_mode(True)
integrator.set_maximum_step_size(0.001)

In [ ]:
# simulate
context = simulator.get_mutable_context()
x0 = np.array([0., 0., 3., 0.])

# since the ZOH block at t=0 returns its internal state, I have to compute the first feedback "offline"
state_c = context.get_mutable_continuous_state_vector()
state_c.SetFromVector(x0)
state_d = context.get_mutable_discrete_state_vector()
u0 = np.empty(pwa.nu)
controller._DoCalcVectorOutput(context, x0, None, u0)
state_d.SetFromVector(u0)

# run sim
simulator.StepTo(3.)

In [ ]:
# visualize
viz = PlanarRigidBodyVisualizer(tree, xlim=[-1, 1], ylim=[-2.5, 3.])
viz.fig.set_size_inches(10, 5)
ani = viz.animate(state_log, 30, repeat=True)
plt.close(viz.fig)
HTML(ani.to_html5_video())